# Assignment 2 

**Group Members:** Don , Umais Siddiqui

Test

# Analysis of Employees working on Projects 

The dataset I have chosen is real Employee data from projects within one department with code 635 and their relationship with employees in other departments that worked on the project with them.

# Data Collection

The data has been collected from a SQL Server source. Below are some queries that were used to format the data for this graph



```
CREATE TABLE #EmployeeProjects (Id int IDENTITY(1,1),ProjectInformationId int, SBUCode varchar(6),Employee nvarchar(255))
Create Table #ProjectSBUCounts( Id int IDENTITY(1,1), ProjectInformationId int,SBUCode varchar(6),TotalCount int)
Create TABLE #TargetProjects(Id int Identity(1,1), ProjectInformationId int,TotalSBUs int)
Create Table #Nodes(NodeId int IDENTITY(1,1),NodeValue int, SBUcode nvarchar(255) )
CREATE TABLE #Edges(SourceNode int,TargetNode int, NumberOfProjects int)
CREATE TABLE #FinalEdges(EdgeId int IDENTITY(1,1) ,SourceNode int, TargetNode int, NumberOfProjects int)
CREATE TABLE #Sources(SourceNode int)
CREATE TABLE #Targets(TargetNode int)
```
Below we will be declaring a GML variable that will hold the final gml file for the graph.



```
DECLARE @gml nvarchar(max)

SET @gml='graph<br>[<br>comment "This is a Employee Project Relationship Graph"<br>'
SET @gml=@gml+'directed 0<br>label "Employee Graph"<br>'
```





INSERT INTO #EmployeeProjects
SELECT DISTINCT  PRojectInformationID,e.SBUCode,v.ADUserID FROM PICreditRateInfoView v 
INNER JOIN Employee e
ON v.ADUserID=e.ADUserID

INSERT INTO  #ProjectSBUCounts
SELECT ProjectInformationId,SBUCode,Count(1)
FROM #EmployeeProjects
GROUP BY PRojectInformationId,SBUCode
Having Count(1)>1

INSERT INTO #TargetProjects
SELECT p.ProjectInformationId,Count(1) FROM #ProjectSBUCounts p
INNER JOIN ProjectCodeRequestView pcr
ON p.ProjectInformationId=pcr.ProjectInformationID
WHERE AgressoSbuCode='635'
GROUP BY p.ProjectInformationId
Having Count(1)>1




DECLARE @StartIndex int=1

DECLARE @EndIndex int

SELECT @EndIndex=Count(1) FROM #TargetProjects

WHILE @StartIndex<=@EndIndex
BEGIN

INSERT INTO #Nodes
SELECT EmployeeId,p.SBUCode
FROM
(SELECT e.EmployeeID,e.SBUCode FROM PICreditRateInfoView v
Inner Join Employee e
ON v.EmployeeID=e.EmployeeID
WHERE ProjectInformationID in (Select ProjectInformationID FROM #TargetProjects WHERE Id=@StartIndex

) and v.[Status]='A'
GROUP BY e.EmployeeID,e.SBUCode
) p
LEFT JOIN #Nodes n
ON p.EmployeeID=n.NodeValue
WHERE n.NodeValue is null

--At this point The Current Project Employee Relationship Can be Made with Other SBU Employee or the relationship can be made stronger

INSERT INTO #Sources
SELECT e.EmployeeID  FROM PICreditRateInfoView v
Inner Join Employee e
ON v.EmployeeID=e.EmployeeID
WHERE ProjectInformationID in (Select ProjectInformationID FROM #TargetProjects WHERE Id=@StartIndex) and SBUCode ='635' and v.[Status]='A'
GROUP BY e.EmployeeID

Insert INTO #Targets
SELECT e.EmployeeID FROM PICreditRateInfoView v
Inner Join Employee e
ON v.EmployeeID=e.EmployeeID
WHERE ProjectInformationID in (Select ProjectInformationID FROM #TargetProjects WHERE Id=@StartIndex) and SBUCode !='635' and v.[Status]='A'
GROUP BY e.EmployeeID


INSERT INTO #Edges
SELECT SOurceNode,TargetNode,1 FROM #Sources,#Targets





 Set @StartIndex=@StartIndex+1
END

INSERT INTO #FinalEdges
SELECT SourceNode,TargetNode,Count(1)
FROM #Edges
GROUP BY SourceNode,TargetNode
--Setting the Starting Index for the Nodes
--Looping through the Nodes
SET @StartIndex=1

SELECT @EndIndex=Max(NodeId) FROM #Nodes
DECLARE @nodeid int
WHILE @StartIndex<=@EndIndex
BEGIN
SELECT @nodeid=NodeValue FROM #Nodes WHERE NodeId=@StartIndex
SET @gml=@gml+'node<br>[<br>id '+CONVERT(nvarchar,@nodeid)+'<br>label "'+CONVERT(nvarchar,@nodeid)+'"<br>]<br>' 
SET @StartIndex=@StartIndex+1
END
 Set @StartIndex=1
SELECT @EndIndex=Max(NodeId) FROM #Nodes
DECLARE @SourceValue int,@TargetValue int
WHILE @StartIndex<=@EndIndex
BEGIN
SELECT @SourceValue=SourceNode FROM #FinalEdges WHERE EdgeId=@StartIndex
SELECT @TargetValue=TargetNode FROM #FinalEdges WHERE EdgeId=@StartIndex
SET @gml=@gml+'edge<br>[<br>source '+CONVERT(nvarchar,@SourceValue)+'<br>target '+CONVERT(nvarchar,@TargetValue)+'<br>]<br>' 
SET @StartIndex=@StartIndex+1
END

SET @gml=@gml+']'

SELECT @gml
SELECT * FROM #Nodes n RIGHT JOIN #FinalEdges f ON n.NodeValue =f.SourceNode WHERE n.NodeId is null


DROP TABLE #EmployeeProjects
DROP TABLE #ProjectSBUCounts
DROP TABLE #TargetProjects
DROP TABLE #Nodes
DROP TABLE #Edges
DROP TABLE #FinalEdges
DROP TABLE #Targets
DROP TABLE #Sources

# END SQL

In [ ]:
from pyvis.network import Network
import networkx as nx


gml_graph=networkx.read_gml('./Employees_Projects.gml')


nt.from_nx(gml_graph)
nt.show("Employee_Projects.html")
HTML(filename="./Employee_Projects.html")



In [ ]:
gml_graph.number_of_edges()




251

In [ ]:
nx.average_node_connectivity(gml_graph)

0.7361593625498007

In [ ]:
try:
  nx.diameter(gml_graph)
except:
  print("Undirected Graph")

Undirected Graph
